In [ ]:
! pip install tensorflow

# 1.Import Liberaries

In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np
from matplotlib import pyplot as plt

In [ ]:

# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

# 2. Data Set


In [ ]:

data_dir = 'Project7'

In [ ]:
os.listdir('project7')


In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

# 3. Data Preprocessing and Showing images

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('Project7')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:

batch = data_iterator.next()

In [ ]:
code= {"Cardboard" : 0 , "Glass" :1 , "Paper" : 2 ,"Vegetation" : 3 }

def getcode(n):
    for x,y in code.items() :
        if n==y:
            return x

In [ ]:
fig, ax = plt.subplots(ncols=6, figsize=(20,20))
for idx, img in enumerate(batch[0][:6]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])
    ax[idx].title.set_text(getcode(batch[1][idx]))


In [ ]:

data = data.map(lambda x,y: (x/255, y))
#normalization

In [ ]:

data.as_numpy_iterator().next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# First Model using CNN

In [ ]:
train

In [ ]:
model= Sequential()

In [ ]:
model.add(Conv2D(32, (3,3),  padding='same',activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Dropout(0.25))  # Dropout layer after pooling
model.add(Conv2D(64, (3,3),padding='same' , activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))  # Another dropout layer
model.add(Conv2D(128, (3,3),padding='same' , activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))  # And another one
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))  # Dropout before the final layers
model.add(Dense(4, activation='softmax'))


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

# Model Training


In [ ]:

logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# Testing Model

In [ ]:
code = {"Cardboard": 0, "Glass": 1, "Paper": 2, "Vegetation": 3}

def getcode(n):
    for x, y in code.items():
        if n == y:
            return x

num_images = 16  # Total number of images to display
num_rows = 4
num_cols = 4

fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

for i, batch in enumerate(test.take(1)):  # Assuming you want to take only one batch
    images, labels = batch  # Assuming each batch contains images and labels
    for j in range(num_images):  # Iterate over all images
        image = images[j]
        label = labels[j]
        predictions = model.predict(np.expand_dims(image, axis=0))
        confidence = predictions[0][label.numpy()]

        ax = axs[j // num_cols, j % num_cols]
        ax.imshow(image)
        ax.set_title(f"Label: {getcode(label.numpy())}, Confidence: {confidence:.2f}")
        ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
y_pred = model.predict(test)

print('Prediction Shape is {}'.format(y_pred.shape))

In [ ]:
# Convert the predictions to a TensorFlow Dataset
predictions_dataset = tf.data.Dataset.from_tensor_slices(y_pred)


In [ ]:
for prediction in predictions_dataset:
    # Process each prediction
    print(prediction.numpy())


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


folder_path = '1/Prediction'


file_names = os.listdir(folder_path)


file_names = file_names[:15]

num_images = len(file_names)
num_images_per_col = 5
num_cols = 3
num_rows = num_images_per_col


fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))


for i, file_name in enumerate(file_names):

    img_path = os.path.join(folder_path, file_name)


    img = image.load_img(img_path, target_size=(256, 256))  # resize to smaller size


    img_array = image.img_to_array(img)


    img_array /= 255.0  # example normalization


    img_array = np.expand_dims(img_array, axis=0)


    predictions = model.predict(img_array)


    predicted_class_index = np.argmax(predictions)
    confidence = predictions[0][predicted_class_index]


    col = i // num_images_per_col
    row = i % num_images_per_col
    ax = axes[row, col]
    ax.imshow(img)
    ax.set_title(f"{getcode(predicted_class_index)}\nConfidence: {confidence:.2f}")
    ax.axis('off')


plt.tight_layout()
plt.show()


# First Model Evaluation

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:

ModelLoss, ModelAccuracy = model.evaluate(val)

print('validation Loss is {}'.format(ModelLoss))
print('validation Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
ModelLoss, ModelAccuracy = model.evaluate(test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
ModelLoss, ModelAccuracy = model.evaluate(train)

print('Train Loss is {}'.format(ModelLoss))
print('Train Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
from sklearn.metrics import classification_report
# Initialize lists to store the true and predicted labels
y_true_values = []
y_pred_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = model.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Store the true and predicted labels
    y_true_values.extend(y_flat)
    y_pred_values.extend(y_pred)
print(classification_report(y_true_values, y_pred_values, target_names=code.keys()))

In [ ]:
from tensorflow.keras.metrics import Accuracy, Precision, Recall
pre=Precision()
re=Recall()
acc=Accuracy()

In [ ]:
import matplotlib.pyplot as plt

# Initialize lists to store the metrics
precision_values = []
recall_values = []
accuracy_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = model.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Update the metrics
    pre.update_state(y_flat, y_pred)
    re.update_state(y_flat, y_pred)
    acc.update_state(y_flat, y_pred)

    # Store the current values of the metrics
    precision_values.append(pre.result().numpy())
    recall_values.append(re.result().numpy())
    accuracy_values.append(acc.result().numpy())

# Plot the metrics
plt.figure(figsize=(12, 8))
plt.plot(precision_values, label='Precision')
plt.plot(recall_values, label='Recall')
plt.plot(accuracy_values, label='Accuracy')
plt.xlabel('Batch number')
plt.ylabel('Metric value')
plt.legend()
plt.show()

In [ ]:
precision_values = []
recall_values = []
accuracy_values = []
for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = model.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array
pre.update_state(y_flat, y_pred)
re.update_state(y_flat, y_pred)
acc.update_state(y_flat, y_pred)
precision_values.append(pre.result().numpy())
recall_values.append(re.result().numpy())
accuracy_values.append(acc.result().numpy())
print('Precision:', pre.result().numpy())
print('Recall:', re.result().numpy())
print('Accuracy:', acc.result().numpy())


In [ ]:

# Initialize lists to store the true and predicted labels
y_true_values = []
y_pred_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = model.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Store the true and predicted labels
    y_true_values.extend(y_flat)
    y_pred_values.extend(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_true_values, y_pred_values)

# Create DataFrame from confusion matrix
cm_df = pd.DataFrame(cm,
                     index = ['Cardboard', 'Glass', 'Paper', 'Vegetation'],
                     columns = ['Cardboard', 'Glass', 'Paper', 'Vegetation'])

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()


# Second Model using VGG16 for optimization

In [ ]:
from keras.applications.vgg16 import VGG16


# Load pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers of the VGG16 model
for layer in base_model.layers:
    layer.trainable = False

# Create your custom model on top of the VGG16 base model
modelV = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Output layer
])

In [ ]:
# Compile the model
modelV.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
modelV.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
# Train the model
hist = modelV.fit(train, epochs=20, validation_data=val, callbacks=[early_stopping])

# Second Model Evaluation

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:

ModelLoss, ModelAccuracy = modelV.evaluate(val)

print('validation Loss is {}'.format(ModelLoss))
print('validation Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
ModelLoss, ModelAccuracy = modelV.evaluate(test)
print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
# Initialize lists to store the true and predicted labels
y_true_values = []
y_pred_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = modelV.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Store the true and predicted labels
    y_true_values.extend(y_flat)
    y_pred_values.extend(y_pred)
print(classification_report(y_true_values, y_pred_values, target_names=code.keys()))

In [ ]:
pre = tf.keras.metrics.Precision()
re = tf.keras.metrics.Recall()
acc = tf.keras.metrics.Accuracy()
precision_values = []
recall_values = []
accuracy_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = modelV.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Update the metrics
    pre.update_state(y_flat, y_pred)
    re.update_state(y_flat, y_pred)
    acc.update_state(y_flat, y_pred)

    # Store the current values of the metrics
    precision_values.append(pre.result().numpy())
    recall_values.append(re.result().numpy())
    accuracy_values.append(acc.result().numpy())

# Plot the metrics
plt.figure(figsize=(12, 8))
plt.plot(precision_values, label='Precision')
plt.plot(recall_values, label='Recall')
plt.plot(accuracy_values, label='Accuracy')
plt.xlabel('Batch number')
plt.ylabel('Metric value')
plt.legend()
plt.show()

In [ ]:
precision_values = []
recall_values = []
accuracy_values = []
for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = modelV.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array
pre.update_state(y_flat, y_pred)
re.update_state(y_flat, y_pred)
acc.update_state(y_flat, y_pred)
precision_values.append(pre.result().numpy())
recall_values.append(re.result().numpy())
accuracy_values.append(acc.result().numpy())
print('Precision:', pre.result().numpy())
print('Recall:', re.result().numpy())
print('Accuracy:', acc.result().numpy())


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
# Initialize lists to store the true and predicted labels
y_true_values = []
y_pred_values = []

for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = modelV.predict(x)
    y_flat = y.flatten() if len(y.shape) > 1 else y

    # Ensure y_pred has the same shape as y_flat
    y_pred = y_pred[:, :y_flat.shape[0]]  # Truncate y_pred to match the length of y_flat
    y_pred = np.argmax(y_pred, axis=1)  # Convert y_pred to a 1D array

    # Store the true and predicted labels
    y_true_values.extend(y_flat)
    y_pred_values.extend(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_true_values, y_pred_values)

# Create DataFrame from confusion matrix
cm_df = pd.DataFrame(cm,
                     index = ['Cardboard', 'Glass', 'Paper', 'Vegetation'],
                     columns = ['Cardboard', 'Glass', 'Paper', 'Vegetation'])

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()


# test

In [ ]:
code = {"Cardboard": 0, "Glass": 1, "Paper": 2, "Vegetation": 3}

def getcode(n):
    for x, y in code.items():
        if n == y:
            return x

num_images = 16  # Total number of images to display
num_rows = 4
num_cols = 4

fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

for i, batch in enumerate(test.take(1)):  # Assuming you want to take only one batch
    images, labels = batch  # Assuming each batch contains images and labels
    for j in range(num_images):  # Iterate over all images
        image = images[j]
        label = labels[j]
        predictions = modelV.predict(np.expand_dims(image, axis=0))
        confidence = predictions[0][label.numpy()]

        ax = axs[j // num_cols, j % num_cols]
        ax.imshow(image)
        ax.set_title(f"Label: {getcode(label.numpy())}, Confidence: {confidence:.2f}")
        ax.axis('off')

plt.tight_layout()
plt.show()


# Prediction

In [ ]:

folder_path = '1/prediction'

# Get a list of all the file names in the folder
file_names = os.listdir(folder_path)

# Display only the first 15 images
file_names = file_names[:15]

# Calculate the number of rows and columns for subplots
num_images = len(file_names)
num_images_per_col = 5  # Number of images per column
num_cols = 3  # Round up for columns
num_rows = num_images_per_col  # Each column will have 5 images

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

# Iterate through each image file
for i, file_name in enumerate(file_names):
    # Construct the full path to the image file
    img_path = os.path.join(folder_path, file_name)

    # Load the image with the right target size for your model
    img = image.load_img(img_path, target_size=(256, 256))  # resize to smaller size

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Normalize the image array (if required)
    img_array /= 255.0  # example normalization

    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the image
    predictions = modelV.predict(img_array)

    # Get the predicted class index and corresponding confidence probability
    predicted_class_index = np.argmax(predictions)
    confidence = np.max(predictions)

    # Plot the image and predicted title in a subplot
    col = i // num_images_per_col
    row = i % num_images_per_col
    ax = axes[row, col]
    ax.imshow(img)
    ax.set_title(f"{getcode(predicted_class_index)}\nConfidence: {confidence:.2f}")
    ax.axis('off')

# Adjust layout and display the subplots
plt.tight_layout()
plt.show()


 # Estimated Weights for Each Category

In [ ]:
# Define the class
class_name = "Cardboard"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001
# Process each image file
for image_file in image_files[:10]:  # Limit to first 10 images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)
    # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)



    # Display the original image and the edges side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edges, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
# Define the class
class_name = "Cardboard"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001
# Initialize a variable to store the total weight of all images
total_weight = 0

# Process each image file
for image_file in image_files[:]:  # Process all images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)

    # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)


    # Add the edge weight to the total weight
    total_weight += object_weight_kg

    print(f"Weight of edge pixels in {image_file}:", object_weight_kg)

# Print the total weight of all images
print("Total weight of Cardboard class", total_weight)

In [ ]:
# Define the class
class_name = "Glass"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001
# Process each image file
for image_file in image_files[:10]:  # Limit to first 10 images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)

   # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)




    # Display the original image and the edges side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edges, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
# Define the class
class_name = "Paper"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001

# Process each image file
for image_file in image_files[:10]:  # Limit to first 10 images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)

 # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)

    # Display the original image and the edges side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edges, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
# Define the class
class_name = "Paper"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5

# Initialize a variable to store the total weight of all images
total_weight = 0
pixel_to_kg_conversion_factor = 0.0001
# Process each image file
for image_file in image_files[:]:  # Process all images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)
 # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)


    # Add the edge weight to the total weight
    total_weight += object_weight_kg

    print(f"Weight of edge pixels in {image_file}:", object_weight_kg)

# Print the total weight of all images
print("Total weight of paper class", total_weight)



In [ ]:
# Define the class
class_name = "Vegetation"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001
# Process each image file
for image_file in image_files[:10]:  # Limit to first 10 images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)

    # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    print(f"Estimated weight of object in Kg {image_file}:", object_weight_kg)



    # Display the original image and the edges side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edges, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
# Define the class
class_name = "Vegetation"

# Define the directory where the images are stored
directory = 'Project7'

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Define the lower and upper threshold for Canny edge detection
lower = 50
upper = 150

# Define the kernel size for Gaussian blurring
kernel_size = 5
pixel_to_kg_conversion_factor = 0.0001
# Initialize a variable to store the total weight of all images
total_weight = 0

# Process each image file
for image_file in image_files[:]:  # Process all images
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, lower, upper)
        # Find contours in the edge-detected image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the largest contour
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Calculate the area of the largest contour
    contour_area = cv2.contourArea(largest_contour)

    # Calculate the weight of the object using the area and the conversion factor
    object_weight_kg = contour_area * pixel_to_kg_conversion_factor

    # Add the object weight to the total weight
    total_weight+= object_weight_kg

    print(f"Estimated weight of object in {image_file}:", object_weight_kg)

# Print the total weight of all objects in all images
print("Total estimated weight of Vegetation class in Kg:", total_weight)

In [ ]:

# Define the class
class_name = "paper"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Process each image file
for image_file in image_files[:10]:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Use the mask to separate the foreground from the background
    foreground = cv2.bitwise_and(image, image, mask=mask)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))

    # Display the original image, the foreground, and the background side by side
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(132), plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
    plt.title('Foreground'), plt.xticks([]), plt.yticks([])
    plt.subplot(133), plt.imshow(cv2.cvtColor(background, cv2.COLOR_BGR2RGB))
    plt.title('Background'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Define the class
class_name = "Paper"

# Define the directory where the images are stored
directory = 'Project7'

# Calibration factor: weight per pixel in kilograms
weight_per_pixel = 0.001  # Example value, replace with your calibration data

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Initialize a variable to store the total weight
total_pixel_weight = 0

# Process each image file
for image_file in image_files:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Calculate the weight for the foreground in pixels
    foreground_pixel_weight = cv2.countNonZero(mask)

    # Update the total pixel weight
    total_pixel_weight += foreground_pixel_weight

# Convert the total pixel weight to kilograms
total_weight_kg = total_pixel_weight * weight_per_pixel

# Print the total weight in kilograms
print("Total weight of all images in class paper:", total_weight_kg, "kg")


In [ ]:
# Define the class
class_name = "Cardboard"

# Define the directory where the images are stored
directory = "Project7"

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Process each image file
for image_file in image_files[:10]:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Use the mask to separate the foreground from the background
    foreground = cv2.bitwise_and(image, image, mask=mask)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))

    # Display the original image, the foreground, and the background side by side
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(132), plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
    plt.title('Foreground'), plt.xticks([]), plt.yticks([])
    plt.subplot(133), plt.imshow(cv2.cvtColor(background, cv2.COLOR_BGR2RGB))
    plt.title('Background'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Define the class
class_name = "Cardboard"

# Define the directory where the images are stored
directory = 'Project7'

# Calibration factor: weight per pixel in kilograms (example value, replace with your calibration data)
weight_per_pixel = 0.001  # Adjust this value based on your actual calibration

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Initialize a variable to store the total weight
total_pixel_weight = 0

# Process each image file
for image_file in image_files:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Calculate the weight for the foreground in pixels
    foreground_pixel_weight = cv2.countNonZero(mask)

    # Update the total pixel weight
    total_pixel_weight += foreground_pixel_weight

# Convert the total pixel weight to kilograms
total_weight_kg = total_pixel_weight * weight_per_pixel

# Print the total weight in kilograms
print("Total weight of all images in class Cardboard:", total_weight_kg, "kg")


In [ ]:
# Define the class
class_name = "Vegetation"

# Define the directory where the images are stored
directory = 'Project7'

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Process each image file
for image_file in image_files[:10]:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Use the mask to separate the foreground from the background
    foreground = cv2.bitwise_and(image, image, mask=mask)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))

    # Display the original image, the foreground, and the background side by side
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(132), plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
    plt.title('Foreground'), plt.xticks([]), plt.yticks([])
    plt.subplot(133), plt.imshow(cv2.cvtColor(background, cv2.COLOR_BGR2RGB))
    plt.title('Background'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Define the class
class_name = "Vegetation"

# Define the directory where the images are stored
directory = 'Project7'

# Calibration factor: weight per pixel in kilograms (example value, replace with your calibration data)
weight_per_pixel = 0.001  # Adjust this value based on your actual calibration

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Initialize a variable to store the total weight
total_pixel_weight = 0

# Process each image file
for image_file in image_files:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Calculate the weight for the foreground in pixels
    foreground_pixel_weight = cv2.countNonZero(mask)

    # Update the total pixel weight
    total_pixel_weight += foreground_pixel_weight

# Convert the total pixel weight to kilograms
total_weight_kg = total_pixel_weight * weight_per_pixel

# Print the total weight in kilograms
print("Total weight of all images in vegetation Cardboard:", total_weight_kg, "kg")


In [ ]:
# Define the class
class_name = "Glass"

# Define the directory where the images are stored
directory = 'Project7'

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Process each image file
for image_file in image_files[:10]:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Use the mask to separate the foreground from the background
    foreground = cv2.bitwise_and(image, image, mask=mask)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))

    # Display the original image, the foreground, and the background side by side
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(132), plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
    plt.title('Foreground'), plt.xticks([]), plt.yticks([])
    plt.subplot(133), plt.imshow(cv2.cvtColor(background, cv2.COLOR_BGR2RGB))
    plt.title('Background'), plt.xticks([]), plt.yticks([])

    plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Define the class
class_name = "Glass"

# Define the directory where the images are stored
directory = 'Project7'

# Calibration factor: weight per pixel in kilograms (example value, replace with your calibration data)
weight_per_pixel = 0.001  # Adjust this value based on your actual calibration

# Get the list of image files for this class
image_files = os.listdir(os.path.join(directory, class_name))

# Initialize a variable to store the total weight
total_pixel_weight = 0

# Process each image file
for image_file in image_files:
    # Load the image
    image = cv2.imread(os.path.join(directory, class_name, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary mask
    _, mask = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Calculate the weight for the foreground in pixels
    foreground_pixel_weight = cv2.countNonZero(mask)

    # Update the total pixel weight
    total_pixel_weight += foreground_pixel_weight

# Convert the total pixel weight to kilograms
total_weight_kg = total_pixel_weight * weight_per_pixel

# Print the total weight in kilograms
print("Total weight of all images in class Glass:", total_weight_kg, "kg")


# Thank You